In [1]:
%run notebook_setup.ipynb

In [2]:
%vault from pubmed_derived_data import literature, reliable_article_types

Imported:

 - `literature` (904B0F94)
 - `reliable_article_types` (5D584CB5)

at Wednesday, 05. Aug 2020 01:51

In [3]:
from helpers import mermaid

In [4]:
pubmed_matches_count = len(literature)
in_pmc_count = sum(literature.has_pmc)
full_text_count = sum(literature.has_full_text == True)
abstract_only_count = sum((~literature.abstract.isnull()) & (~(literature.has_full_text == True)))
article_types_count = len(reliable_article_types)

In [5]:
%%mermaid
graph TD
    classDef data fill:#ffe49f,stroke:#333,stroke-width:1px;
    classDef integration fill:#b6d7ab,stroke:#333,stroke-width:1px;
    classDef analysis fill:#93c482,stroke:#333,stroke-width:1px,padding:0;
    SEARCH[fa:fa-search {pubmed_matches_count} PubMed results]
    SEARCH:::data-->PMC[fa:fa-file-text {in_pmc_count} articles in PubMed Central]:::data
    PMC:::data-->FULL_TEXT[fa:fa-align-justify {full_text_count} articles with full-text]:::data
    SEARCH-->ABSTRACTS[fa:fa-font {abstract_only_count} with abstracts only]:::data
    FULL_TEXT-->COMBINED[fa:fa-plus-circle Combined dataset]:::integration
    ABSTRACTS-->COMBINED
    ABSTRACTS-->CLINICAL{{{{fa:fa-stethoscope Disease and species}}}}:::analysis
    COMBINED-->TYPES{{{{fa:fa-shapes {article_types_count} articles with determined type}}}}:::analysis
    COMBINED-->REPOS{{{{fa:fa-code code and data repositories}}}}:::analysis
    COMBINED-->OMICS{{{{fa:fa-dna omics}}}}:::analysis
    COMBINED-->TRENDS{{{{fa:fa-calendar-alt phrase trends}}}}:::analysis